In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

VER = 172
POSTFIX = ''

USE = 'orders'

CT=50

In [ ]:
import pandas as pd, cudf, numpy as np, gc
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt

In [ ]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
df = df.sort_values(['session','ts'])
df = df.groupby('session').agg('last').reset_index()
df = df[['session','aid']]
df.columns = ['user','last']
print( df.shape )
df.head()

In [ ]:
users = pd.read_parquet(f'../../data/train_with_features/train_v{VER}{POSTFIX}.pqt')
print( users.shape )
users.head()

In [ ]:
%%time
users2 = users[['user','item']].copy()
users2 = users2.merge(df.to_pandas(),on='user',how='left')
print( users2.shape )
users2.head()

In [ ]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
df = df.sort_values(['session','ts'])
df = df.groupby('session').agg('first').reset_index()
df = df[['session','aid']]
df.columns = ['user','first']
print( df.shape )
df.head()

In [ ]:
%%time
users2 = users2.merge(df.to_pandas(),on='user',how='left')
print( users2.shape )
users2.head()

In [ ]:
df = cudf.read_parquet('../../data/train_data/test.parquet')
df = df.sort_values(['session','ts']).reset_index(drop=True)

df['mx'] = df.groupby('session').ts.transform('max')
df['mx'] = df.mx - df.ts
df = df.loc[df.mx > 60*60*24]

df = df.groupby('session').agg('last').reset_index()
df = df[['session','aid']]
df.columns = ['user','last2']
print( df.shape )
df.head()

In [ ]:
%%time
users2 = users2.merge(df.to_pandas(),on='user',how='left')
users2['last2'] = users2['last2'].fillna(users2['last']).astype('int32')
print( users2.shape )
users2.head()

In [ ]:
users2['last'].isna().sum()

In [ ]:
users2['last2'].isna().sum()

In [ ]:
users2['first'].isna().sum()

# Matrix Factorization

In [ ]:
%%time
x = users2['first'].values
y = users2['last'].values

In [ ]:
embed = np.load('../../data/item_user_features/item_embed_32.npy')
z = np.sqrt( np.sum(embed * embed,axis=1) )
embed = embed/ np.reshape(z,(-1,1))
embed.shape
#(1855603, 32)

In [ ]:
%%time
e1 = embed[x,]
e2 = embed[y,]

In [ ]:
%%time
z = np.sum( e1*e2,axis=1 )

In [ ]:
users2['z10'] = z

In [ ]:
%%time
x = users2['last2'].values
#y = users2['last'].values

In [ ]:
%%time
e1 = embed[x,]
#e2 = embed[y,]

In [ ]:
%%time
z = np.sum( e1*e2,axis=1 )

In [ ]:
users2['z11'] = z

# Add Item Features

In [ ]:
df = pd.read_parquet('../../data/item_user_features/item40.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

In [ ]:
%%time
users2 = users2.merge(df, left_on='item', right_index=True, how='left')
print( users2.shape )
users2.head()

In [ ]:
df = pd.read_parquet('../../data/item_user_features/item41.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

In [ ]:
%%time
users2 = users2.merge(df, left_on='item', right_index=True, how='left')
print( users2.shape )
users2.head()

In [ ]:
COLS = [f'{x.replace("12","13")}' for x in df.columns]
df.columns = COLS
print( COLS )

In [ ]:
%%time
users2 = users2.merge(df, left_on='last', right_index=True, how='left')
print( users2.shape )
users2.head()

# Add User Features

In [ ]:
df = pd.read_parquet('../../data/item_user_features/user20.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

In [ ]:
%%time
users2 = users2.merge(df, left_on='user', right_index=True, how='left')
print( users2.shape )
users2.head()

In [ ]:
df = pd.read_parquet('../../data/item_user_features/user21.pqt')
for c in df.columns:
    if str(df[c].dtype)=='int32':
        print(c, df[c].dtype)
        df[c] = df[c].astype('float32')
df.head()

In [ ]:
%%time
users2 = users2.merge(df, left_on='user', right_index=True, how='left')
print( users2.shape )
users2.head()

# EDA Columns

In [ ]:
for c in users2.columns:
    print(c,users2[c].dtype)

In [ ]:
for c in users2.columns[:5]:
    print(c,', ',end='')

In [ ]:
for c in users2.columns[5:]:
    print(c,', ',end='')
    users[c] = users2[c].values
del users2
gc.collect()

In [ ]:
for c in users.columns:
    t = str(users[c].dtype)
    if (t=='int64'): 
        users[c] = users[c].astype('int32')
        print('convert',c,'to int32')
    if (t=='float64')|(t=='Int32')|(t=='Float32'):
        users[c] = users[c].astype('float32')
        print('convert',c,'to float32')

In [ ]:
print( users.shape )
users.head()

In [ ]:
%%time
users.to_parquet(f'../../data/train_with_features/train_v{VER}.pqt',index=False)